In [1]:
import tensorflow as tf
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
%pylab
%matplotlib inline

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
def swish(features):
  # pylint: disable=g-doc-args
  """Computes the Swish activation function: `x * sigmoid(x)`.
  Source: "Searching for Activation Functions" (Ramachandran et al. 2017)
  https://arxiv.org/abs/1710.05941
  Args:
    features: A `Tensor` representing preactivation values.
    name: A name for the operation (optional).
  Returns:
    The activation value.
  """
  # pylint: enable=g-doc-args
  features = ops.convert_to_tensor(features, name="features")
  return features * math_ops.sigmoid(features)

In [3]:
# 위에설명이 나와있지만 입력이 784x10000 이며 10개로 softmax분류를 한다.
x_input = tf.placeholder(tf.float32, [None, 784])

In [4]:
def batch_norm(x):
	epsilon = 1e-3
	batch_mean, batch_var = tf.nn.moments(x, [0])
	return tf.nn.batch_normalization(x, batch_mean, batch_var, None, None, epsilon)

W_1=tf.Variable(tf.truncated_normal([784,392],stddev=0.01),name='weight1')
B_1=tf.Variable(tf.constant(1.0,shape=[392]),name='bias1')
layer1=tf.nn.elu(tf.matmul(x_input,W_1)+B_1)

W_2=tf.Variable(tf.truncated_normal([392,196],stddev=0.01),name='weight2')
B_2=tf.Variable(tf.constant(1.0,shape=[196]),name='bias2')
layer1=batch_norm(layer1)
layer2=tf.nn.elu(tf.matmul(layer1,W_2)+B_2)

W_3=tf.Variable(tf.truncated_normal([196,98],stddev=0.01),name='weight3')
B_3=tf.Variable(tf.constant(1.0,shape=[98]),name='bias3')
layer2=batch_norm(layer2)
layer3=tf.nn.elu(tf.matmul(layer2,W_3)+B_3)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
with tf.device('/gpu:0'):
    W = tf.Variable(tf.truncated_normal([98, 10],stddev=0.1),name='fcwWeight')
    b = tf.Variable(tf.truncated_normal([10],stddev=0.1),name='fcBias')
    y=tf.nn.softmax(tf.matmul(layer3,W)+b)
    y_=tf.placeholder(tf.float32,[None,10])
    

    cost=tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))
    cost_summ = tf.summary.scalar("cost", cost)

    train_step=tf.train.AdamOptimizer(0.0001).minimize(cost)

    sess=tf.InteractiveSession()


In [6]:
tf.global_variables_initializer().run()

In [7]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [8]:
for _ in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x_input: batch_xs, y_: batch_ys})


In [9]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("accuracy:",sess.run(accuracy, feed_dict={x_input: mnist.test.images, y_: mnist.test.labels}))

accuracy: 0.9575
